# 1. Importação das bilbiotecas necessárias



In [ ]:
# Importando o pandas
import pandas as pd

# Importando biblioteca para trabalhar com o MongoDB
import pymongo as pym

# Importando o numpy para trabalhar com cálculos matemáticos
import numpy as np

#Importando o seaborn e o matplotlib para análises gráficas
import seaborn as sns
import matplotlib.pyplot as plt

# 2. Carga do arquivo e dados iniciais

In [ ]:
# Lendo o arquivo CSV disponível nos microdados do Censo da Educação Superior e gravando
# ele em um DataFrame
cursos = pd.read_csv('../data/SUP_CURSO_2019.CSV', sep="|", encoding="latin-1")

In [ ]:
# Verificando a quantidade de linhas e colunas do arquivo 
cursos.shape

In [ ]:
# Verificando as 5 primeiras linhas do arquivo
cursos.head()

In [ ]:
# Selecionando somente as colunas necessárias descritas na documentação do trabalho: 
# COD_IES, CO_CURSO,NO_CURSO, TP_SITUACAO, TP_GRAU_ACADEMICO, TP_MODALIDADE_ENSINO
# TP_NIVEL_ACADEMICO, IN_GRATUITO, NU_CARGA_HORARIA 

cursos = cursos[{'CO_IES','CO_CURSO','NO_CURSO','TP_SITUACAO','TP_GRAU_ACADEMICO',
                'TP_MODALIDADE_ENSINO','TP_NIVEL_ACADEMICO','IN_GRATUITO',
                 'NU_CARGA_HORARIA'}]

In [ ]:
# Verificando novamente a quantidade de linhas e colunas do arquivo
# após a seleção de somente as colunas necessárias
cursos.shape

In [ ]:
# Verificando o tipo de cada coluna
cursos.dtypes

In [ ]:
# Verificando a 5 primeiras linhas do novo arquivo
cursos.head()

# 3. Tratando os dados do dataset

In [ ]:
## Verificando a quantidade de valores nulos por coluna
cursos.isnull().sum()

In [ ]:
# No arquivo do Censo, cada atributo está armazenado como número 
# então preciso fazer o tratamento desses dados conforme dicionário 
# também disponível no Censo 

In [ ]:
# Alterar os valores nulos por "NÃO INFORMADO"
cursos['TP_GRAU_ACADEMICO'].fillna(value='Não informado', inplace=True)

In [ ]:
# 1) Alterando o tipo dos dados int, para object (string)
cursos['TP_SITUACAO'] = cursos['TP_SITUACAO'].apply(str)
cursos['TP_NIVEL_ACADEMICO'] = cursos['TP_NIVEL_ACADEMICO'].apply(str)
cursos['TP_MODALIDADE_ENSINO'] = cursos['TP_MODALIDADE_ENSINO'].apply(str)
cursos['TP_GRAU_ACADEMICO'] = cursos['TP_GRAU_ACADEMICO'].apply(str)
cursos['IN_GRATUITO'] = cursos['IN_GRATUITO'].apply(str)

In [ ]:
# Alterando os valores por suas respectivas categorias  
# 2) TP_SITUACAO: 1: Em atividade; 2:Extinto; 3: Em extinção.  
cursos.loc[cursos['TP_SITUACAO'] == '1', 'TP_SITUACAO'] = 'Em atividade'
cursos.loc[cursos['TP_SITUACAO'] == '2', 'TP_SITUACAO'] = 'Extinto'
cursos.loc[cursos['TP_SITUACAO'] == '3', 'TP_SITUACAO'] = 'Em extinção'

# 3) TP_NIVEL_ACADEMICO: 1: Graduação; 2:Sequencial
cursos.loc[cursos['TP_NIVEL_ACADEMICO'] == '1', 'TP_NIVEL_ACADEMICO'] = 'Graduação'
cursos.loc[cursos['TP_NIVEL_ACADEMICO'] == '2', 'TP_NIVEL_ACADEMICO'] = 'Sequencial'

# 4) TP_MODALIDADE_ENSINO: 1: Presencial; 2:EAD
cursos.loc[cursos['TP_MODALIDADE_ENSINO'] == '1', 'TP_MODALIDADE_ENSINO'] = 'Presencial'
cursos.loc[cursos['TP_MODALIDADE_ENSINO'] == '2', 'TP_MODALIDADE_ENSINO'] = 'EAD'

# 5) TP_GRAU_ACADEMICO: 1: Bacharelado; 2: Licenciatura; 3: Tecnológico; 4: Bacharelado e Licenciatura. 
cursos.loc[cursos['TP_GRAU_ACADEMICO'] == '1.0', 'TP_GRAU_ACADEMICO'] = 'Bacharelado'
cursos.loc[cursos['TP_GRAU_ACADEMICO'] == '2.0', 'TP_GRAU_ACADEMICO'] = 'Licenciatura'
cursos.loc[cursos['TP_GRAU_ACADEMICO'] == '3.0', 'TP_GRAU_ACADEMICO'] = 'Tecnológico'
cursos.loc[cursos['TP_GRAU_ACADEMICO'] == '4.0', 'TP_GRAU_ACADEMICO'] = 'Bacharelado e Licenciatura'

# 6) IN_GRATUITO: 0: Não; 1: Sim. 
cursos.loc[cursos['IN_GRATUITO'] == '0', 'IN_GRATUITO'] = 'Não'
cursos.loc[cursos['IN_GRATUITO'] == '1', 'IN_GRATUITO'] = 'Sim'

In [ ]:
cursos.groupby(by='IN_GRATUITO').count()

In [ ]:
# Resultado do dataset após categorização das colunas necessaŕias: 

cursos.head(5)

# 4. Análise dos dados de cursos

In [ ]:
print("Quantidade de linhas: ", cursos.shape[0])
print("Quantidade de colunas: ",cursos.shape[1])

In [ ]:
# Descrição dos dados
cursos.describe()

In [ ]:
print("Quantidade de cursos por modalidade")
sns.countplot(x='TP_MODALIDADE_ENSINO', data=cursos)

In [ ]:
print("Quantidade de cursos por situação")
sns.countplot(x='TP_SITUACAO', data=cursos)

In [ ]:
print("Quantidade de cursos por grau acadêmico")
sns.countplot(x='TP_GRAU_ACADEMICO', data=cursos)

In [ ]:
print("Quantidade de cursos por nível")
sns.countplot(x='TP_NIVEL_ACADEMICO', data=cursos)

In [ ]:
print("Quantidade de cursos gratuítos e pagos")
sns.countplot(x='IN_GRATUITO', data=cursos)

# 5. Envio dos dados para o mongoDB

In [ ]:
# Criando a conexão com o mongoDB
client = pym.MongoClient('mongodb://localhost:27017/')

# Acessando a base de dados criada para o teste
db = client['censo']

# Acessando a coleção curso dentro da base de dados censo  
collection_cursos=  db['cursos']

In [ ]:
# Transformando o DataSet dos cursos em um dicionario com o objetivo
# de salvar ele no MongoDB, pois o banco não aceita estruturas do tipo DataFrame
cursos_dict = cursos.to_dict('records')

In [ ]:
# Enviando os dados para o MongoDB
collection_cursos.insert_many(cursos_dict)